# generate topic based on human text

In [ ]:

import json
import time
from datetime import datetime
from tqdm import tqdm
from openai import OpenAI


client = OpenAI(api_key="")


def get_prompt(text):
    return f"""You are a topic summarizer.
Summarize the following paragraph into a very short topic (max 40 words).
Avoid using full sentences. Be specific.
Text:
{text}
Return only the topic without any explanation."""


def generate_topic(text):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": get_prompt(text)}],
            temperature=0.3,
            max_tokens=30
        )
        return response.choices[0].message.content.strip().strip('"')
    except Exception as e:
        print(f"❌ Error generating topic: {e}")
        return ""

# input json file containing human written texts
input_path = ""
output_path = ""


with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)


for item in tqdm(data, desc="Generating topics"):
    if not item.get("topic"):
        item["topic"] = generate_topic(item["text"])
        time.sleep(1)


with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("all topics have saved")


🧠 Generating topics: 100%|██████████| 2100/2100 [31:39<00:00,  1.11it/s]


✅ 所有 topic 已生成并保存至 merged_corpus_all_genres_withTopic.json


# using openai to generate same length text

In [ ]:

import os
import json
import time
from tqdm import tqdm
from openai import OpenAI

# input your api key
client = OpenAI(api_key="")


file_path = ""  # input json file containing human text and realted topic
output_path = ""
processed_count = 0

）
def generate_openai_text(topic, word_count):
    prompt = f"""You are a writing assistant.

Write an original passage on the topic: '{topic}'. It should be approximately {word_count} words long. Be clear and human-like. Avoid copying or referencing specific texts.

⚠️ Do not include or repeat the topic or instructions in your output. Return only the generated passage text."""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=min(word_count * 2, 2048)
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ OpenAI generation error for topic '{topic[:30]}...': {e}")
        return ""


with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

if os.path.exists(output_path):
    with open(output_path, "r", encoding="utf-8") as f:
        processed_data = json.load(f)


    processed_indices = {}
    for idx, item in enumerate(processed_data):

        text = item.get("text", "").strip()
        if text:
            processed_indices[text] = idx


else:
    processed_data = []
    processed_indices = {}



def save_current_progress():
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=2)
    print(f" saved {len(processed_data)} samples in {output_path}")

try:
    for i, item in enumerate(tqdm(data, desc="🧠 Generating with OpenAI")):
        topic = item.get("topic", "").strip()
        text = item.get("text", "").strip()

        if not topic or not text:
            continue


        if text in processed_indices:

            existing_idx = processed_indices[text]
            existing_item = processed_data[existing_idx]

            if not existing_item.get("ai_generated", {}).get("openai"):
                word_count = len(text.split())
                ai_text = generate_openai_text(topic, word_count)


                if "ai_generated" not in existing_item:
                    existing_item["ai_generated"] = {}
                existing_item["ai_generated"]["openai"] = ai_text
                processed_count += 1


                save_current_progress()
        else:

            word_count = len(text.split())


            if "ai_generated" not in item:
                item["ai_generated"] = {}


            if not item["ai_generated"].get("openai"):
                ai_text = generate_openai_text(topic, word_count)
                item["ai_generated"]["openai"] = ai_text
                processed_count += 1

            processed_data.append(item)
            processed_indices[text] = len(processed_data) - 1


            save_current_progress()


        if (i + 1) % 50 == 0:
            print(f"have processed {i + 1}/{len(data)} samples，newly generated {processed_count} AI texts")

except KeyboardInterrupt:
    save_current_progress()
except Exception as e:
    print(f"{e}")
    save_current_progress()
    raise

# 最终保存（以防万一）
save_current_progress()

print(f"all done! processed total {len(data)} samples，generated {processed_count} AI texts")

已读取输出文件中的 2100 条记录


🧠 Generating with OpenAI:   0%|          | 0/2100 [00:00<?, ?it/s]

已处理 50/2100 条目，新生成 0 条AI文本
已处理 100/2100 条目，新生成 0 条AI文本
已处理 150/2100 条目，新生成 0 条AI文本
已处理 200/2100 条目，新生成 0 条AI文本
已处理 250/2100 条目，新生成 0 条AI文本
已处理 300/2100 条目，新生成 0 条AI文本
已处理 350/2100 条目，新生成 0 条AI文本
已处理 400/2100 条目，新生成 0 条AI文本
已处理 450/2100 条目，新生成 0 条AI文本
已处理 500/2100 条目，新生成 0 条AI文本
已处理 550/2100 条目，新生成 0 条AI文本
已处理 600/2100 条目，新生成 0 条AI文本
已处理 650/2100 条目，新生成 0 条AI文本
已处理 700/2100 条目，新生成 0 条AI文本
已处理 750/2100 条目，新生成 0 条AI文本


🧠 Generating with OpenAI:  38%|███▊      | 799/2100 [00:03<00:06, 212.96it/s]

✅ 已保存 2100 条样本到：/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withOpenai.json
✅ 已保存 2100 条样本到：/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withOpenai.json
已处理 800/2100 条目，新生成 2 条AI文本
✅ 已保存 2100 条样本到：/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withOpenai.json
✅ 已保存 2100 条样本到：/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withOpenai.json


...

🧠 Generating with OpenAI: 100%|█████████▉| 2099/2100 [2:00:34<00:06,  6.97s/it]

✅ 已保存 2100 条样本到：/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withOpenai.json


🧠 Generating with OpenAI: 100%|██████████| 2100/2100 [2:00:40<00:00,  3.45s/it]

✅ 已保存 2100 条样本到：/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withOpenai.json
已处理 2100/2100 条目，新生成 1302 条AI文本


✅ 已保存 2100 条样本到：/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withOpenai.json
✅ 处理完成！共处理 2100 条样本，新生成 1302 条AI文本


# use 5 ai detector(4 api and one fine-tuned model) to detect texts

In [ ]:
# only pengram
import json
import requests
from tqdm import tqdm

# === Load Pangram API key ===
api_keys = {
    "pengram": ""
}

# === Pangram 检测函数 ===
def detect_ai_pangram(text):
    if not text.strip():
        return None
    headers = {
        "Content-Type": "application/json",
        "x-api-key": api_keys["pengram"]
    }
    payload = {"text": text}
    try:
        response = requests.post("https://text.api.pangramlabs.com", headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()

        likelihood = result.get("ai_likelihood")
        prediction = result.get("prediction")

        return {
            # "verdict": 1 if isinstance(likelihood, (float, int)) and likelihood > 0.5 else 0,
            "ai_likelihood": likelihood,
            "prediction": prediction
        }
    except Exception as e:
        return {"error": str(e)}

# === 加载输入数据 ===
input_path = ""
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# === conduct Pangram detection ===
for item in tqdm(data, desc="Running Pangram detection"):
    human_text = item.get("text", "")
    item["pengram_human_verdict"] = detect_ai_pangram(human_text)

    ai_texts = item.get("ai_generated", {})
    item["pengram_ai_verdicts"] = {}
    for model_name, text in ai_texts.items():
        item["pengram_ai_verdicts"][model_name] = detect_ai_pangram(text)

# === save result ===
output_path = input_path.replace(".json", "_pangram_only.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ Pangram-only detection completed. Results saved to:", output_path)


Running Pangram detection: 100%|██████████| 2100/2100 [34:58<00:00,  1.00it/s]


✅ Pangram-only detection completed. Results saved to: /content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withOpenai_all_detectors_pangram_only.json


In [ ]:
# only originality
import json
import requests
from tqdm import tqdm

# === API key for Originality.ai ===
api_keys = {
    "originality": ""
}

# === Originality.ai wrapper that extracts full classification + confidence
def detect_ai_originality(text):
    if not text.strip():
        return None
    try:
        headers = {
            "X-OAI-API-KEY": api_keys["originality"],
            "Content-Type": "application/json"
        }
        payload = {
            "check_ai": True,
            "check_plagiarism": False,
            "check_facts": False,
            "check_readability": False,
            "check_grammar": False,
            "check_contentOptimizer": False,
            "storeScan": False,
            "aiModelVersion": "lite",
            "content": text
        }
        response = requests.post("https://api.originality.ai/api/v3/scan", headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        classification = result.get("results", {}).get("ai", {}).get("classification", {})
        confidence = result.get("results", {}).get("ai", {}).get("confidence", {})
        return {
            "classification": {
                "AI": classification.get("AI"),
                "Original": classification.get("Original")
            },
            "confidence": {
                "AI": confidence.get("AI"),
                "Original": confidence.get("Original")
            }
        }
    except Exception as e:
        return {"error": str(e)}

# === Load input data
input_path = ""
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# === Run detection using only Originality.ai
for item in tqdm(data, desc="Running Originality.ai detection"):
    human_text = item.get("text", "")
    item["human_verdict"] = {
        "originality": detect_ai_originality(human_text)
    }

    ai_texts = item.get("ai_generated", {})
    item["ai_verdicts"] = {}
    for model_name, text in ai_texts.items():
        item["ai_verdicts"][model_name] = {
            "originality": detect_ai_originality(text)
        }

# === Save output
output_path = input_path.replace(".json", "_originality_pangram.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ Originality-only detection completed. Results saved to:", output_path)



Retrying Originality.ai 422 errors if any: 100%|██████████| 600/600 [00:42<00:00, 14.02it/s]

✅ Retry complete. Results saved to: /content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_all_genres_generated_openai_withVerdict_originality_fixed.json


In [ ]:
# only GPTzero
import json
import requests
from tqdm import tqdm

# === Load GPTZero API Key ===
api_keys = {
    "gptzero": ""
}

# === GPTZero detector wrapper (returns dict with key info)
def detect_ai_gptzero(text):
    if not text.strip():
        return None
    try:
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "x-api-key": api_keys["gptzero"]
        }
        payload = {
            "document": text,
            "multilingual": False
        }
        response = requests.post(
            "https://api.gptzero.me/v2/predict/text",
            headers=headers,
            json=payload
        )
        response.raise_for_status()
        doc = response.json().get("documents", [{}])[0]
        return {
            "average_generated_prob": doc.get("average_generated_prob"),
            # "predicted_class": doc.get("document_classification"),
            # "confidence_score": doc.get("confidence_score")
        }
    except Exception as e:
        return {"error": str(e)}

# === Load input data
input_path = ""
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# === Run GPTZero detection
for item in tqdm(data, desc="Running GPTZero detection"):
    human_text = item.get("text", "")

    # ✅ 在已有 human_verdict 上添加 gptzero 字段
    item.setdefault("human_verdict", {})["gptzero"] = detect_ai_gptzero(human_text)

    ai_texts = item.get("ai_generated", {})
    item.setdefault("ai_verdicts", {})
    for model_name, text in ai_texts.items():
        item["ai_verdicts"].setdefault(model_name, {})["gptzero"] = detect_ai_gptzero(text)

# === Save output
output_path = input_path.replace(".json", "_gptzero_dict.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ GPTZero-only detection completed. Results saved to:", output_path)


Running GPTZero detection: 100%|██████████| 600/600 [17:20<00:00,  1.73s/it]

✅ GPTZero-only detection completed. Results saved to: /content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_all_genres_generated_openai_withVerdict_gptzero_dict.json


In [ ]:
# only zerogpt

import json
import requests
from tqdm import tqdm

# === Load ZeroGPT API Key ===
api_keys = {
    "zerogpt": ""
}

# === ZeroGPT detector wrapper (structured return)
def detect_ai_zerogpt(text):
    if not text.strip():
        return None
    try:
        headers = {
            "ApiKey": api_keys["zerogpt"],
            "Content-Type": "application/json"
        }
        payload = {"input_text": text}
        response = requests.post(
            "https://api.zerogpt.com/api/detect/detectText",
            headers=headers,
            json=payload
        )
        response.raise_for_status()
        result = response.json().get("data", {})
        return {
            "is_gpt_generated": result.get("is_gpt_generated"),
            "is_human_written": result.get("is_human_written"),
            # "feedback_message": result.get("feedback_message"),
            # "words_count": result.get("words_count")
        }
    except Exception as e:
        return {"error": str(e)}

# === Load input data
input_path = ""

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)[:10]

# === Run ZeroGPT detection

for item in tqdm(data, desc="Running ZeroGPT detection"):
    human_text = item.get("text", "")

    # ✅ 安全追加到已有的 human_verdict
    item.setdefault("human_verdict", {})["zerogpt"] = detect_ai_zerogpt(human_text)

    ai_texts = item.get("ai_generated", {})
    item.setdefault("ai_verdicts", {})  # 确保外层存在
    for model_name, text in ai_texts.items():
        # ✅ 安全追加到已有的 ai_verdicts[model_name]
        item["ai_verdicts"].setdefault(model_name, {})["zerogpt"] = detect_ai_zerogpt(text)


# === Save output
output_path = input_path.replace(".json", "_zerogpt_only.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ ZeroGPT-only detection completed. Results saved to:", output_path)


Running ZeroGPT detection: 100%|██████████| 10/10 [00:46<00:00,  4.66s/it]

✅ ZeroGPT-only detection completed. Results saved to: /content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_all_genres_generated_openai_withVerdict_zerogpt_only.json


In [ ]:
# only use roBERTa

import json
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm

# === Load RoBERTa-based OpenAI Detector ===
MODEL_NAME = "roberta-base-openai-detector"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
detector = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).eval()

# === RoBERTa detector: returns probability of AI-generated
def detect_ai_roberta(text):
    if not text.strip():
        return None
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        logits = detector(**inputs).logits
    prob_ai = torch.softmax(logits, dim=-1)[0, 1].item()
    return prob_ai  # 返回 AI 概率

# === Load input data
input_path = ""
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# === Run RoBERTa-only detection
for item in tqdm(data, desc="Running RoBERTa detection"):
    human_text = item.get("text", "")
    item.setdefault("human_verdict", {})["roberta-base-detector"] = detect_ai_roberta(human_text)

    ai_texts = item.get("ai_generated", {})
    item.setdefault("ai_verdicts", {})
    for model_name, text in ai_texts.items():
        item["ai_verdicts"].setdefault(model_name, {})["roberta-base-detector"] = detect_ai_roberta(text)

# === Save output
output_path = input_path.replace(".json", "_roberta_only.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ RoBERTa-only detection completed. Results saved to:", output_path)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Running RoBERTa detection: 100%|██████████| 600/600 [24:48<00:00,  2.48s/it]

✅ RoBERTa-only detection completed. Results saved to: /content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_all_genres_generated_openai_withVerdict_roberta_only.json


In [ ]:

# merge all ai-detector, and remain default prob
# Multi-detector AI text analysis with default prob
import json
import requests
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm

# === Load API keys ===
api_keys = {
    "pengram": "",
    "originality": "",
    "gptzero": ""
}

# === Pangram 检测函数 ===
def detect_ai_pangram(text):
    if not text.strip():
        return None
    headers = {
        "Content-Type": "application/json",
        "x-api-key": api_keys["pengram"]
    }
    payload = {"text": text}
    try:
        response = requests.post("https://text.api.pangramlabs.com", headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()

        likelihood = result.get("ai_likelihood")
        prediction = result.get("prediction")

        return {
            "ai_likelihood": likelihood,
            "prediction": prediction
        }
    except Exception as e:
        return {"error": str(e)}

# === Originality.ai wrapper ===
def detect_ai_originality(text):
    if not text.strip():
        return None
    try:
        headers = {
            "X-OAI-API-KEY": api_keys["originality"],
            "Content-Type": "application/json"
        }
        payload = {
            "check_ai": True,
            "check_plagiarism": False,
            "check_facts": False,
            "check_readability": False,
            "check_grammar": False,
            "check_contentOptimizer": False,
            "storeScan": False,
            "aiModelVersion": "lite",
            "content": text
        }
        response = requests.post("https://api.originality.ai/api/v3/scan", headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        classification = result.get("results", {}).get("ai", {}).get("classification", {})
        confidence = result.get("results", {}).get("ai", {}).get("confidence", {})
        return {
            "classification": {
                "AI": classification.get("AI"),
                "Original": classification.get("Original")
            },
            "confidence": {
                "AI": confidence.get("AI"),
                "Original": confidence.get("Original")
            }
        }
    except Exception as e:
        return {"error": str(e)}

# === GPTZero detector wrapper ===
def detect_ai_gptzero(text):
    if not text.strip():
        return None
    try:
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "x-api-key": api_keys["gptzero"]
        }
        payload = {
            "document": text,
            "multilingual": False
        }
        response = requests.post(
            "https://api.gptzero.me/v2/predict/text",
            headers=headers,
            json=payload
        )
        response.raise_for_status()
        doc = response.json().get("documents", [{}])[0]
        return {
            "average_generated_prob": doc.get("average_generated_prob")
        }
    except Exception as e:
        return {"error": str(e)}

# === Load RoBERTa-based OpenAI Detector ===
MODEL_NAME = "roberta-base-openai-detector"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
detector = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).eval()

# === RoBERTa detector function ===
def detect_ai_roberta(text):
    if not text.strip():
        return None
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        logits = detector(**inputs).logits
    prob_ai = torch.softmax(logits, dim=-1)[0, 1].item()
    return prob_ai

# === Load input data ===
input_path = ""
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 统计信息
total_items = len(data)
skipped_items = 0
processed_items = 0

# === Run all detectors ===
for item in tqdm(data, desc="Running all AI detectors"):
    # 检查是否已有检测结果，如果有则跳过
    if "human_verdict" in item and "ai_verdicts" in item and all(model in item["ai_verdicts"] for model in item.get("ai_generated", {})):
        skipped_items += 1
        continue

    human_text = item.get("text", "")

    # Initialize verdict structure if needed
    item.setdefault("human_verdict", {})

    # Add results from each detector
    item["human_verdict"]["pengram"] = detect_ai_pangram(human_text)
    item["human_verdict"]["originality"] = detect_ai_originality(human_text)
    item["human_verdict"]["gptzero"] = detect_ai_gptzero(human_text)
    item["human_verdict"]["roberta-base-detector"] = detect_ai_roberta(human_text)

    # Process AI-generated texts
    ai_texts = item.get("ai_generated", {})
    item.setdefault("ai_verdicts", {})

    for model_name, text in ai_texts.items():
        item["ai_verdicts"].setdefault(model_name, {})
        item["ai_verdicts"][model_name]["pengram"] = detect_ai_pangram(text)
        item["ai_verdicts"][model_name]["originality"] = detect_ai_originality(text)
        item["ai_verdicts"][model_name]["gptzero"] = detect_ai_gptzero(text)
        item["ai_verdicts"][model_name]["roberta-base-detector"] = detect_ai_roberta(text)

    processed_items += 1

# === Save output ===
output_path = input_path.replace(".json", "_all_detectors.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"✅ Multi-detector analysis completed.")
print(f"Total items: {total_items}")
print(f"Skipped items (already processed): {skipped_items}")
print(f"Processed items: {processed_items}")
print(f"Results saved to: {output_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Running all AI detectors: 100%|██████████| 2100/2100 [3:26:12<00:00,  5.89s/it]


✅ Multi-detector analysis completed.
Total items: 2100
Skipped items (already processed): 608
Processed items: 1492
Results saved to: /content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withOpenai_all_detectors.json
